In [17]:
from ollama import Client
client = Client(host='http://localhost:11434')
import yaml

In [21]:
def get_summary(book_info):
  query = "I have up to the end of " + book_info['bookmark']
  query += " in the book '" +  book_info['book'] + "' by " +  book_info['author']
  query += ". Can you summarize the character of " +  book_info['character']
  query += " for me? I don't want any spoilers."
  response = client.chat(model='llama3', messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

In [22]:
def spoiler_check(book_info, summary):
  query = "Read the following summary of " +  book_info['character']
  query += " from '" +  book_info['book'] + "' by " +  book_info['author'] + ": '" + summary
  query += "'. The reader has only read up to the end of " +  book_info['bookmark']
  query += ". Considering this, check to see whether the following spoiler is present: '"
  query += book_info['spoiler'] + "'."
  response = client.chat(model='llama3', messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

In [20]:
with open('spoilers/pride.yml', 'r') as file:
    pride = yaml.safe_load(file)
pride

{'book': 'Pride and Prejudice',
 'author': 'Jane Austen',
 'bookmark': 'Volume I',
 'character': 'Mr. Wickham',
 'spoiler': 'Mr. Wickham betrays the trust of the Bennet family by eloping with Lydia Bennet, the youngest daughter.'}

In [23]:
pride_and_prejudice_summary = get_summary(pride)
print(pride_and_prejudice_summary)

You've only read up to the end of Volume I, which means you're just getting started on your journey through this beloved novel!

As you continue reading, you'll have the pleasure of encountering Charles Wickham, a charming and roguish officer in the militia. Here's what you can expect:

* Mr. Wickham is a charismatic individual who has a way with words and a quick wit.
* He's described as being handsome, which adds to his charm.
* He's also known for being somewhat of a ladies' man, having a reputation for being flirtatious and possibly even unscrupulous when it comes to the fairer sex.
* Despite his questionable character, Mr. Wickham is able to present himself in a flattering light, making him seem like an attractive and desirable suitor.

That's all I can reveal without giving away any spoilers! You'll get to know more about Mr. Wickham as you continue reading the novel, but for now, let's just say that he's someone who will likely play an important role in the story ahead.


In [24]:
pride_and_prejudice_spolier_check = spoiler_check(pride, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

Since the reader has only read up to the end of Volume I, it's likely that the spoiler "Mr. Wickham betrays the trust of the Bennet family by eloping with Lydia Bennet, the youngest daughter." is NOT present in the summary.

In fact, the summary suggests that the reader will get to know more about Mr. Wickham as they continue reading the novel, which implies that this event has not yet occurred. Therefore, it's safe to say that this spoiler is still ahead, waiting to be revealed as the reader continues their journey through the story!
